sequence to sequence task

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
import csv
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import SGDClassifier
import torch.nn.functional as F
torch.manual_seed(1)

In [ ]:
#my implementation
#imports in cell 1
#import data
embeddings_dict = {}
with open("glove.6B.50d.txt", 'r', encoding="utf-8") as f:
  for line in f:
    values = line.split()
    word = values[0]
    vector = np.asarray(values[1:], "float32")
    embeddings_dict[word] = vector
train_data = pd.read_csv("hw_3_train_data.csv")

In [ ]:
x=[label.split() for label in train_data['utterances']]
#y = [label.split() for label in train_data['Core Relations']]
y = [label.split() for label in train_data['IOB Slot tags']]

test_data = pd.read_csv("test_data.csv")
x_test=[label.split() for label in test_data['utterances']]

#get vocab
target_vocab = {"<PAD>":0}
index = 1
for sentence in x:
  for word in sentence:
    if word in target_vocab.keys():
      continue
    else:
      target_vocab[word] = index
      index = index +1

for sentence in x_test:
  for word in sentence:
    if word in target_vocab.keys():
      continue
    else:
      target_vocab[word] = index
      index = index +1

#embeddings
matrix_len = len(target_vocab)
weights_matrix = np.zeros((matrix_len, 50))
words_found = 0

for i, word in enumerate(target_vocab):
    try: 
        weights_matrix[i] = embeddings_dict[word]
        words_found += 1
    except KeyError:
       #continue
       weights_matrix[i] = np.random.normal(scale=0.6, size=(50, ))

#labels
tag_vocab = {"<PAD>":0}
index = 1
for i in y:
  for tag_set in y:
    for tag in tag_set:
      if tag in tag_vocab.keys():
        continue
      else:
        tag_vocab[tag] = index
        index = index +1
#binarizer = MultiLabelBinarizer()
#binarizer.fit(y)
#y = binarizer.transform(y)



x_train, x_dt, y_train, y_dt = train_test_split(x, y, test_size = .1, random_state = 0)
print ((weights_matrix.shape))
EMBEDDING_DIM = 50
HIDDEN_DIM = 50
print (tag_vocab)
print (len(target_vocab))
print (x_train[0], y_train[0])
print (embeddings_dict['word'])

(1482, 50)
{'<PAD>': 0, 'O': 1, 'B_char': 2, 'B_movie': 3, 'I_movie': 4, 'I_char': 5, 'B_person': 6, 'I_person': 7, 'B_director': 8, 'I_director': 9, 'B_release_year': 10, 'I_release_year': 11, 'B_location': 12, 'B_country': 13, 'I_country': 14, 'B_producer': 15, 'I_producer': 16, 'B_cast': 17, 'I_cast': 18, 'B_subject': 19, 'B_genre': 20, 'I-movie': 21, 'B_mpaa_rating': 22, 'I_mpaa_rating': 23, 'B_language': 24, 'I_language': 25, 'I_genre': 26, 'I_subject': 27}
1482
['what', 'movies', 'are', 'in', 'bengali'] ['O', 'O', 'O', 'O', 'B_language']
[-0.1643     0.15722   -0.55021   -0.3303     0.66463   -0.1152
 -0.2261    -0.23674   -0.86119    0.24319    0.074499   0.61081
  0.73683   -0.35224    0.61346    0.0050975 -0.62538   -0.0050458
  0.18392   -0.12214   -0.65973   -0.30673    0.35038    0.75805
  1.0183    -1.7424    -1.4277     0.38032    0.37713   -0.74941
  2.9401    -0.8097    -0.66901    0.23123   -0.073194  -0.13624
  0.24424   -1.0129    -0.24919   -0.06893    0.70231   -0.

In [ ]:
#padding of the sequences

In [ ]:
class LSTMTagger(nn.Module):

    def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size, weights_matrix):
        super(LSTMTagger, self).__init__()
        self.hidden_dim = hidden_dim

        #self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)
        #addtion
        weights_matrix = torch.from_numpy(weights_matrix)
        
        num_embeddings, embedding_dim = weights_matrix.shape
        self.word_embeddings = nn.Embedding(num_embeddings, embedding_dim)
        self.word_embeddings.load_state_dict({'weight': weights_matrix})
        self.word_embeddings.weight.requires_grad = False


        self.lstm = nn.LSTM(embedding_dim, embedding_dim, bidirectional = True)

        self.hidden2tag = nn.Linear(embedding_dim * 2, tagset_size)
        #drop out
        self.dropout = torch.nn.Dropout(0.5)

    def forward(self, sentence):
        embeds = self.word_embeddings(sentence)
        #embeds = self.dropout(embeds)
        lstm_out, _ = self.lstm(embeds.view(len(sentence), 1, -1))
        #lstm_out = self.dropout(lstm_out)
        tag_space = self.hidden2tag(lstm_out.view(len(sentence), -1))
        tag_scores = F.log_softmax(tag_space, dim=1)
        return tag_scores

In [ ]:
#model
def prepare_sequence(seq, to_ix, pad_length):
    idxs = [to_ix[w] for w in seq]
    tensor = torch.tensor(idxs, dtype=torch.long)
    if len(tensor) > pad_length:
      tensor = tensor[0:pad_length]
    if len(tensor) < pad_length:
      tensor = F.pad(input = tensor,pad=(0, pad_length-len(tensor)),  mode='constant', value = 0)
    return tensor

model = LSTMTagger(EMBEDDING_DIM, HIDDEN_DIM, len(target_vocab), len(tag_vocab), weights_matrix)
loss_function = nn.NLLLoss()
#loss_function = nn.CrossEntropyLoss()
#
optimizer = optim.SGD(model.parameters(), lr=0.1)
#optimizer = optim.Adam(model.parameters(), lr = 0.01 )


#previously tried 8 12 16 21
pad_length = 22


for epoch in range(100): 
    print ("epoch:", epoch)
    sentences = []
    targets = []
    for i in range(0, len(x_train)):
        sentences.append(prepare_sequence(x_train[i], target_vocab, pad_length))
        #print (y_train.to_list())
        targets.append(prepare_sequence(y_train[i], tag_vocab, pad_length))

    for i  in range(0, len(x_train)):

        model.zero_grad()

        sentence_in = sentences[i]
        target = targets[i]

        tag_scores = model(sentence_in)

        loss = loss_function(tag_scores, target)
        loss.backward()
        optimizer.step()



#testing

In [ ]:
y_pred = []
#x_dt
x_val = x_train
y_val = y_train
count = 0
right = 0
with torch.no_grad():
  for i in range(0, len(y_val)):
    pred = ((model(prepare_sequence(x_val[i], target_vocab, pad_length)).numpy())) #np.argmax
    temp= []
    for row in pred:
      best_tag = np.argmax(row)
      best_tag = [key for key, value in tag_vocab.items() if value == best_tag]
      temp.append(best_tag[0])
    #print (temp)
    pred = temp
    y_pred.append(pred)
    #print (y_pred[i])
  for i in range(0,len(y_pred)):
    if len(y_pred[i]) > len(y_val[i]):
      y_pred[i] = y_pred[i][0:len(y_val[i])]
    if y_pred[i] == ():
      y_pred[i] = ('none',)
    if y_pred[i] == y_val[i]:
      right = right +1
    #else:
    #print (y_pred[i], y_val[i])
    count = count + 1
    #if len(y_pred[i]) != len(y_val[i]):
      #print (i, y_pred[i], y_val[i])
  #ex = 3
  #print (y_pred[ex], y_val[ex])
  print (right)
  print (count)
  print (right/count)

y_pred = []
x_val = x_dt
y_val = y_dt
count = 0
right = 0
with torch.no_grad():
  for i in range(0, len(y_val)):
    pred = ((model(prepare_sequence(x_val[i], target_vocab, pad_length)).numpy())) #np.argmax
    temp= []
    for row in pred:
      best_tag = np.argmax(row)
      best_tag = [key for key, value in tag_vocab.items() if value == best_tag]
      temp.append(best_tag[0])
    #print (temp)
    pred = temp
    y_pred.append(pred)
    #print (y_pred[i])
  for i in range(0,len(y_pred)):
    if len(y_pred[i]) > len(y_val[i]):
      y_pred[i] = y_pred[i][0:len(y_val[i])]
    if y_pred[i] == ():
      y_pred[i] = ('none',)
    if y_pred[i] == y_val[i]:
      right = right +1
    #else:
    #print (y_pred[i], y_val[i])
    count = count + 1
    #if len(y_pred[i]) != len(y_val[i]):
      #print (i, y_pred[i], y_val[i])
  #ex = 3
  #print (y_pred[ex], y_val[ex])
  print (right)
  print (count)
  print (right/count)

2023
2027
0.9980266403552047
177
226
0.7831858407079646


In [ ]:
#exporting test data
import csv
y_pred = []
x_val = x_test
with torch.no_grad():
  for i in range(0, len(x_val)):
    pred = ((model(prepare_sequence(x_val[i], target_vocab, pad_length)).numpy())) #np.argmax
    temp= []
    for row in pred:
      best_tag = np.argmax(row)
      best_tag = [key for key, value in tag_vocab.items() if value == best_tag]
      temp.append(best_tag[0])
    #print (temp)
    pred = temp
    y_pred.append(pred)
    #print (y_pred[i])

with open('model_output.csv', 'w', newline = '') as csvfile:
    writer = csv.writer(csvfile, delimiter = ',', lineterminator='\n')
    writer.writerow(['Id', 'Predicted'])
    count = 0
    for i in range(0,len(y_pred)):
      y_pred[i] = y_pred[i][0 : len(x_test[i])]
      for j in y_pred[i]:
        out = j
        writer.writerow((count, out))
        count += 1
    #buffering size until error is found
    #writer.writerow((count, out))